# Notebook to map respondent ID to entity ID

This notebook provides a simple "first draft" of mapping of `respondent_id`'s from the historical
DBF based FERC data to `entity_id`'s in the new XBRL based data. To do this, this notebook
will use the years of data that FERC has migrated to the new XBRL format. Each filing they
have migrated contains the `respondent_id` in the file name, and the `entity_id` embedded in
the filings.

The first step is to extract the migrated filings to a SQLite database to make the entity ID's
accessible. These filings can be downloaded [here](https://ferc.gov/filing-forms/eforms-refresh/migrated-data-downloads).
There's not data for every filer included in each year of data, so using the entire set of years will
provide the best results. To create the SQLite database, extract the downloaded zip files to a single
directory, then use the FERC XBRL extractor tool with the following command:

```
xbrl_extract {path_to_filing_directory} ferc1.sqlite
```

In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from fuzzywuzzy import fuzz

The only data needed to perform the mapping are the `filing_name`, and `entity_id` columns from the `identificiation_001_duration` table.

In [ ]:
engine = sa.create_engine("sqlite:///ferc1.sqlite")

# Select RespondentLegalName as well for convenience
id_table = pd.read_sql(
    "SELECT filing_name, entity_id, RespondentLegalName FROM identification_001_duration",
    engine,
    parse_dates=["start_date", "end_date"]
)

The `respondent_id` is embedded in each filing name with the format `{UtilityName}-{respondent_id}-{year}{quarter}{form_number}`.
The first step is extract that ID, then drop duplicate pairs (same pairs will exist for different years).

In [ ]:
id_table["respondent_id"] = pd.to_numeric(
    id_table["filing_name"].str.extract(r'.+-(\d+)-.+').loc[:,0]
)
map_table = (
    id_table.drop("filing_name", axis=1)
    .drop_duplicates(subset=["entity_id", "respondent_id"])
    .convert_dtypes()
    .sort_values(by=["respondent_id"])
)

Save the mapping to a CSV file.

In [ ]:
map_table.to_csv("respondent_map.csv", index=False)

Check for any `entity_id`'s that map to multiple `respondent_id`'s. These will need to be analyzed
further to identify if there are any mistakes.

In [ ]:
map_table.loc[map_table["entity_id"].duplicated(), :]

## Supplement missing `respondent_id`'s with string matching
There are 257 unique `respondent_id -> entity_id` pairs. It's hard to say exactly how many there *should* be, because the set of unique `respondent_id`'s varies a fair amount from year to year. FERC has indicated that not all filings are included in the migrated data, so it is likely that there are missing `respondent_id`'s. To attempt to supplement this mapping, we can try to use the respondent legal names to try and find missing pairs. First, get all `respondent_id`'s from the raw DBF database produced by `ferc1_to_sqlite` in PUDL.

In [ ]:
ferc1_db_path = "/home/zach/catalyst/workspace/sqlite/ferc1.sqlite"
pudl_engine = sa.create_engine("sqlite:///" + ferc1_db_path)
inspector = sa.inspect(pudl_engine)

# Get all respondent_id's by report_year
df = pd.DataFrame(columns=["report_year", "respondent_id"])
for tbl in inspector.get_table_names():
    cols = [col['name'] for col in inspector.get_columns(tbl)]
    if ("report_year" in cols) & ("respondent_id" in cols):
        df = pd.concat([df, pd.read_sql(f"SELECT DISTINCT report_year, respondent_id FROM {tbl}", pudl_engine)])

df = df.drop_duplicates()

# Find respondent_id's that don't exist in migrated XBRL data
missing_ids = set(df["respondent_id"]) - set(map_table["respondent_id"])
print(len(missing_ids))

There are 116 ID's that are missing in the XBRL data, which can certainly be mapped manually, but it might be easier to try and use some fuzzy string matching to help out. There's a table provided by FERC that maps between the new `entity_id`'s and respondent names, which can be downloaded [here](https://www.ferc.gov/media/ferc-cid-listing). This will be used to do the string matching. Many of the missing ID's also don't appear at all in newer years of data, and quite possibly don't exist in FERC's new Company Identifier system, so it will likely only be a subset of the missing ID's for which there is a match at all.

In [ ]:
# Load CID lookup table
lookup_table = pd.read_excel("FERC_CID_Listing_6-6-2022.xlsx", skiprows=2)
lookup_table

For each missing `respondent_id` find the respondent name that most closely matches the name listed in the DBF database.

In [ ]:
# Read respondent ID table from historical data for respondent names
respondent_names = pd.read_sql("SELECT respondent_id, respondent_name FROM f1_respondent_id", pudl_engine)
respondent_names = respondent_names.set_index("respondent_id")

# Loop through missing IDs and do a fuzzy string match between respondent names and CID lookup table names
new_matches_df = {"entity_id": [], "RespondentLegalName": [], "respondent_id": [], "old_name": []}
for r_id in missing_ids:
    name = respondent_names["respondent_name"][r_id]
    matches = lookup_table["Organization Name"].apply(fuzz.ratio, args=(name,))
    
    if matches.max() > 85:
        # Add row to new dataframe
        new_matches_df["entity_id"].append(lookup_table["CID"][matches.idxmax()])
        new_matches_df["respondent_id"].append(r_id)
        new_matches_df["RespondentLegalName"].append(lookup_table["Organization Name"][matches.idxmax()])
        new_matches_df["old_name"].append(name)

        
        # Print matches for visual inspection
        print(f"{name} -> {lookup_table['Organization Name'][matches.idxmax()]}")

new_matches_df = pd.DataFrame(new_matches_df).set_index("old_name")

I've limited the matches to only those with a fuzzy match ratio greater than 85. This is somewhat arbitrary, but I settled on this by lowering it until all additional matches found were clearly false positives. There are clearly several false positives that need to be manually removed.

In [ ]:
# False positives from visual inspection
false_positives = [
    "IES Utilities Inc.",
    "TXE Electric Company",
    "Bridger Valley Electric Association, Inc.",
    "Flowell Electric Association, Inc.",
]

# The top match for 'Ocean State Power' was 'Ocean State Power II', so I'll manually map this to 'Ocean State Power LLC'
new_matches_df.loc["Ocean State Power"] = {"RespondentLegalName": "Ocean State Power LLC", "entity_id": "C001906", "respondent_id": 124"}

# Drop false positives then get rid of old_name column
new_matches_df = new_matches_df.drop(
    new_matches_df[new_matches_df["old_name"].isin(false_positives)].index
)
new_matches_df = new_matches_df.drop("old_name", axis=1)

Now these new matches can be added to the mapping table and the CSV can be rewritten.

In [ ]:
map_table = pd.concat([map_table, new_matches_df])
map_table.to_csv("respondent_map.csv", index=False)